# Сравнение архитектур CNN на датасете EMNIST (Notebook)

In [ ]:
!pip install torch torchvision matplotlib numpy

In [ ]:
import torch, torchvision, time
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

In [ ]:
# Data transforms
transform = transforms.Compose([transforms.Resize(224), transforms.Grayscale(3), transforms.ToTensor(), transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
# Load EMNIST Balanced
train_ds = datasets.EMNIST('.', split='balanced', train=True, download=True, transform=transform)
test_ds = datasets.EMNIST('.', split='balanced', train=False, download=True, transform=transform)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)
print(len(train_ds), 'train samples')
print(len(test_ds), 'test samples')

In [ ]:
# Example: show 6 samples
fig, axs = plt.subplots(2,3,figsize=(8,4))
for idx, ax in enumerate(axs.flat):
    img,label = train_ds[idx*1000]
    ax.imshow(img.permute(1,2,0).numpy(), cmap='gray')
    ax.set_title(label)
    ax.axis('off')
plt.show()

In [ ]:
# Define model and train 1 epoch
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features,47)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
start = time.time()
model.train()
running_loss=0
for imgs, lbls in train_loader:
    imgs,lbls = imgs.to(device), lbls.to(device)
    optimizer.zero_grad()
    out = model(imgs)
    loss = criterion(out,lbls)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
print('Train Loss:', running_loss/len(train_loader), 'Time:', time.time()-start)